In [1]:
import Eisenia

In [2]:
import BioSequences
import GeneticVariation

In [3]:
import Random

In [4]:
import Dates
import LightGraphs
import Statistics
import StatsBase

In [5]:
LENGTH = 5
k = 3
N_SEQUENCES = 2
Random.seed!(LENGTH)

Random.MersenneTwister(UInt32[0x00000005], Random.DSFMT.DSFMT_state(Int32[-85655800, 1072952617, -522203688, 1073721902, 1588056292, 1072823262, 1930843968, 1073164255, -719121749, 1073022455  …  647343590, 1073165527, 1405924594, 1072769931, 1969171087, 42945320, -226019479, -1469955508, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000

In [6]:
sequences = Vector{BioSequences.FASTA.Record}()
for i in 1:N_SEQUENCES
    seq = BioSequences.randdnaseq(LENGTH)
    id = Random.randstring(LENGTH)
    record = BioSequences.FASTA.Record(id, seq)
    push!(sequences, record)
end
sequences

2-element Array{BioSequences.FASTA.Record,1}:
 BioSequences.FASTA.Record:
   identifier: rb3hy
  description: <missing>
     sequence: GTCTT
 BioSequences.FASTA.Record:
   identifier: Mz1S6
  description: <missing>
     sequence: GTGAC

In [7]:
READ_DEPTH=3

3

In [8]:
observations = Vector{BioSequences.FASTA.Record}()
for sequence in sequences
    for depth in 1:READ_DEPTH
        push!(observations, Eisenia.observe(sequence))
    end
end
observations

6-element Array{BioSequences.FASTA.Record,1}:
 BioSequences.FASTA.Record:
   identifier: wykeFdvYlui2
  description: <missing>
     sequence: GTCTT
 BioSequences.FASTA.Record:
   identifier: yobYAJK5WEzK
  description: <missing>
     sequence: AAGAC
 BioSequences.FASTA.Record:
   identifier: 8kot3XbhBxwh
  description: <missing>
     sequence: GTCTT
 BioSequences.FASTA.Record:
   identifier: stTiiijF3Shc
  description: <missing>
     sequence: GTGAC
 BioSequences.FASTA.Record:
   identifier: ZIeq5YfJjHNd
  description: <missing>
     sequence: GTGAC
 BioSequences.FASTA.Record:
   identifier: vJyYjoRayiBZ
  description: <missing>
     sequence: GTGAC

In [9]:
canonical_kmers, canonical_kmer_counts = Eisenia.count_canonical_kmers(observations, k)

(BioSequences.Kmer{BioSymbols.DNA,3}[AAG, AGA, CAC, GAC, TCA], [3, 3, 3, 6, 3])

In [10]:
stranded_kmer_graph = Eisenia.build_stranded_kmer_graph(canonical_kmers, observations)

{10, 6} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [11]:
filename = "$(hash(BioSequences.FASTA.identifier.(observations))).strand_specific.svg"
Eisenia.plot_stranded_kmer_graph(stranded_kmer_graph, filename=filename)
HTML("""
<image src="$filename" width=50%>
""")

HTML{String}("<image src=\"14324062447322809448.strand_specific.svg\" width=50%>\n")

In [12]:
canonical_kmer_graph = Eisenia.build_canonical_kmer_graph(canonical_kmers, observations)

{5, 4} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [13]:
filename = "$(hash(BioSequences.FASTA.identifier.(observations))).canonical.svg"
Eisenia.plot_canonical_kmer_graph(canonical_kmer_graph, filename=filename)
HTML("""
<image src="$filename" width=50%>
""")

HTML{String}("<image src=\"14324062447322809448.canonical.svg\" width=50%>\n")

In [14]:
Eisenia.report_graph_structure(canonical_kmer_graph)

(sequences = BioSequences.FASTA.Record[BioSequences.FASTA.Record:
   identifier: 1
  description: <missing>
     sequence: AAGAC], variants = GeneticVariation.VCF.Record[])

In [15]:
# return alternate paths along dominant path(s) with coverage info as VCF
# not applicable here

In [55]:
function walk(canonical_kmer_graph, initial_vertex, initial_orientation, visited)
    current_vertex = initial_vertex
    current_orientation = initial_orientation
    walk = []
    done = false
    while !done
        potential_next_steps = []
        current_vertex_coverage = canonical_kmer_graph.vprops[current_vertex][:coverage]
        # need to subset relevant coverage to what is in the graph
        for coverage in current_vertex_coverage
            (observation, (index, orientation)) = coverage
            observation_path = canonical_kmer_graph.gprops[:observed_paths][observation]
            # if true current orientation then we want to go up in trues and down in falses
            if current_orientation
                if orientation
                    next_index = index + 1
                else
                    next_index = index - 1
                end
            else
                if orientation
                    next_index = index - 1
                else
                    next_index = index + 1
                end
            end
            if next_index in eachindex(observation_path)
                next_vertex, next_orientation = observation_path[next_index]
                if !(next_vertex in visited)
                    if next_index < index
                        next_orientation = !next_orientation
                    end
                    push!(potential_next_steps, next_vertex => next_orientation)
                end
            end
        end
        if isempty(potential_next_steps)
            done = true
        else
            step_votes = StatsBase.countmap(potential_next_steps)
            max_count = maximum(values(step_votes))
            all_best_steps = filter(step_vote -> step_vote[2] == max_count, step_votes)
            next_step = first(rand(all_best_steps))
            push!(walk, next_step)
            current_vertex, current_orientation = next_step
            visited = [visited..., current_vertex]
        end
    end
    return walk
end

walk (generic function with 1 method)

In [73]:
function report_graph_structure(canonical_kmer_graph)
    k = canonical_kmer_graph.gprops[:k]
    sequences = BioSequences.FASTA.Record[]
    variants = GeneticVariation.VCF.Record[]
    # return the dominant path(s) as fasta with coverage info
    for (i, connected_component) in enumerate(LightGraphs.connected_components(canonical_kmer_graph))
        visited = Int[]
        maximum_weight = maximum(v -> length(canonical_kmer_graph.vprops[v][:coverage]), connected_component)
        maximum_weight_indices = findall(v -> length(canonical_kmer_graph.vprops[v][:coverage]) == maximum_weight, connected_component)
        initial_vertex = connected_component[rand(maximum_weight_indices)]

        true_walk = walk(canonical_kmer_graph, initial_vertex, true, visited)
        false_walk = walk(canonical_kmer_graph, initial_vertex, false, visited)

        reverse_complemented_false_walk = [vertex => !orientation for (vertex, orientation) in false_walk[end:-1:1]]
        path = [reverse_complemented_false_walk..., initial_vertex => true, true_walk...]
        (vertex, orientation) = first(path)
        kmer = canonical_kmer_graph.gprops[:canonical_kmers][vertex]
        if !orientation
            kmer = BioSequences.reverse_complement(kmer)
        end
        initial_sequence = BioSequences.DNASequence(kmer)
        path_sequence = initial_sequence
        for (vertex, orientation) in path[2:end]
            kmer = canonical_kmer_graph.gprops[:canonical_kmers][vertex]
            if !orientation
                kmer = BioSequences.reverse_complement(kmer)
            end
            sequence = BioSequences.DNASequence(kmer)
            sequence
            sequence_prefix = sequence[1:end-1]
            path_sequence_suffix = path_sequence[end-(k-2):end]
            @assert sequence_prefix == path_sequence_suffix
            push!(path_sequence, sequence[end])
        end
        primary_contig = BioSequences.FASTA.Record("$i", path_sequence)
        push!(sequences, primary_contig)
        visited = vcat(visited, first.(path))
        remaining = [v for v in connected_component if !(v in visited)]
        current_variant = 1
        while !isempty(remaining)
            maximum_weight = maximum(v -> length(canonical_kmer_graph.vprops[v][:coverage]), remaining)
            maximum_weight_indices = findall(v -> length(canonical_kmer_graph.vprops[v][:coverage]) == maximum_weight, remaining)
            initial_vertex = remaining[rand(maximum_weight_indices)]

            true_walk = walk(canonical_kmer_graph, initial_vertex, true, visited)
            false_walk = walk(canonical_kmer_graph, initial_vertex, false, visited)

            reverse_complemented_false_walk = [vertex => !orientation for (vertex, orientation) in false_walk[end:-1:1]]
            path = [reverse_complemented_false_walk..., initial_vertex => true, true_walk...]
            (vertex, orientation) = first(path)
            kmer = canonical_kmer_graph.gprops[:canonical_kmers][vertex]
            if !orientation
                kmer = BioSequences.reverse_complement(kmer)
            end
            initial_sequence = BioSequences.DNASequence(kmer)
            path_sequence = initial_sequence
            for (vertex, orientation) in path[2:end]
                kmer = canonical_kmer_graph.gprops[:canonical_kmers][vertex]
                if !orientation
                    kmer = BioSequences.reverse_complement(kmer)
                end
                sequence = BioSequences.DNASequence(kmer)
                sequence
                sequence_prefix = sequence[1:end-1]
                path_sequence_suffix = path_sequence[end-(k-2):end]
                @assert sequence_prefix == path_sequence_suffix
                push!(path_sequence, sequence[end])
            end
            @show BioSequences.FASTA.Record("$i-$current_variant", path_sequence)
            visited = vcat(visited, first.(path))
            remaining = [v for v in connected_component if !(v in visited)]
            current_variant += 1
        end
    end
    return (sequences = sequences, variants = variants)
end

report_graph_structure (generic function with 1 method)

In [74]:
report_graph_structure(canonical_kmer_graph)

BioSequences.FASTA.Record("$(i)-$(current_variant)", path_sequence) = BioSequences.FASTA.Record:
   identifier: 1-1
  description: <missing>
     sequence: AAGA


(sequences = BioSequences.FASTA.Record[BioSequences.FASTA.Record:
   identifier: 1
  description: <missing>
     sequence: GTGAC], variants = GeneticVariation.VCF.Record[])